<a href="https://colab.research.google.com/github/AfaqSaeed/MergingOpenSfm-clouds/blob/main/Merge_OpenSFM_Point_Clouds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/GoPro data/DL Labelling Data/ExtractFrames
import cv2 
import os
vids=os.listdir(".")
print(vids[0])
cap = cv2.VideoCapture(f"./{vids[0]}")
frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(frames)
for i in range(0,int(frames)):
  ret , img =  cap.read()
  if img is None:
    continue
  if i%6 ==0:
    cv2.imwrite(f"/content/drive/MyDrive/3D Reconstruction/RedPavementFrames/Frame{str(i)}.png",img)
    


/content/drive/MyDrive/GoPro data/DL Labelling Data/ExtractFrames
GX010037.MP4
6113.0


In [ ]:
!pip install plyFile


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load libraries and Import prewritten functions

In [ ]:
import cv2
import json
import pandas as pd 
import numpy as np
from plyfile import PlyData as ply
import os
import struct
from scipy.spatial import KDTree
def read_ply_xyzrgb(filename):
    """ read XYZRGB point cloud from filename PLY file """
    assert(os.path.isfile(filename))
    with open(filename, 'rb') as f:
        plydata = ply.read(f)
        num_verts = plydata['vertex'].count
        rgb = np.zeros(shape=[num_verts, 3], dtype=np.float32)
        xyz = np.zeros(shape=[num_verts, 3], dtype=np.float32)
        
        xyz[:,0] = plydata['vertex'].data['x']
        xyz[:,1] = plydata['vertex'].data['y']
        xyz[:,2] = plydata['vertex'].data['z']
        try :
            rgb[:,0] = plydata['vertex'].data['diffuse_red']
            rgb[:,1] = plydata['vertex'].data['diffuse_green']
            rgb[:,2] = plydata['vertex'].data['diffuse_blue']
        except:
                rgb[:,0] = plydata['vertex'].data['red']
                rgb[:,1] = plydata['vertex'].data['green']
                rgb[:,2] = plydata['vertex'].data['blue']
    return xyz,rgb

def write_pointcloud(xyz_points,rgb_points,filename):

    """ creates a .ply file of the point clouds generated
    """

    assert xyz_points.shape[1] == 3,'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8)*255
    assert xyz_points.shape == rgb_points.shape,'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    fid = open(filename,'wb')
    fid.write(bytes('ply\n', 'utf-8'))
    fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
    fid.write(bytes('element vertex %d\n'%xyz_points.shape[0], 'utf-8'))
    fid.write(bytes('property float x\n', 'utf-8'))
    fid.write(bytes('property float y\n', 'utf-8'))
    fid.write(bytes('property float z\n', 'utf-8'))
    fid.write(bytes('property uchar red\n', 'utf-8'))
    fid.write(bytes('property uchar green\n', 'utf-8'))
    fid.write(bytes('property uchar blue\n', 'utf-8'))
    fid.write(bytes('end_header\n', 'utf-8'))

    # Write 3D points to .ply file
    for i in range(xyz_points.shape[0]):
        fid.write(bytearray(struct.pack("fffccc",xyz_points[i,0],xyz_points[i,1],xyz_points[i,2],
                                        rgb_points[i,0].tobytes(),rgb_points[i,1].tobytes(),
                                        rgb_points[i,2].tobytes())))
    fid.close()

def getRTfromjson(reconstruction,image):
    Rangle_form =np.array( reconstruction['shots'][image]['rotation'],dtype=np.float64)
    R = cv2.Rodrigues(Rangle_form)[0]
    T = np.array(reconstruction['shots'][image]['translation'],dtype=np.float64).reshape(3,1)
    return R,T


def get_K_in_pixel_coordinates(focal, width=None, height=None):
        """The calibration matrix that maps to pixel coordinates.
        Coordinates (0,0) correspond to the center of the top-left pixel,
        and (width - 1, height - 1) to the center of bottom-right pixel.
        You can optionally pass the width and height of the image, in case
        you are using a resized versior of the original image.
        """
        w = width 
        h = height 
        f = focal * max(w, h)
        return np.array([[f, 0, 0.5 * (w - 1)],
                         [0, f, 0.5 * (h - 1)],
                         [0, 0, 1.0]])

def getKD(reconstruction):
    width = reconstruction['cameras']['v2 unknown unknown 1920 1080 perspective 0.0']['width']
    height = reconstruction['cameras']['v2 unknown unknown 1920 1080 perspective 0.0']['height']
    focal = reconstruction['cameras']['v2 unknown unknown 1920 1080 perspective 0.0']['focal']
  
    k1 = reconstruction['cameras']['v2 unknown unknown 1920 1080 perspective 0.0']['k1']
    k2 = reconstruction['cameras']['v2 unknown unknown 1920 1080 perspective 0.0']['k2']
    D =np.array([ k1 , k2 ,0,0,0])

    K = get_K_in_pixel_coordinates(focal,width,height)
    
    return K,D,width,height
 
    print(R1.shape,T1.shape)

    K,D,_,_ = getKD(reconstruction)
    Rt1 = np.concatenate([R1,T1],axis=-1)    
    Rt2 = np.concatenate([R2,T2],axis=-1)
    P1 = np.matmul(K,Rt1)
    P2 = np.matmul(K,Rt2)
    R = R2 - R1
    T = T2 - T1

    if sendRT:
        return K,R1,T1,R2,T2,D

    else:
        return P1,P2

def applyhomogenuos(a,M):
    
    t=np.dot(M,a)

    return t  
def to4D(imagepts):
  
  pad = np.array([[1]]*imagepts.shape[0])
  print(imagepts.shape)
  imagepts = np.concatenate([imagepts,pad],axis=1)  
  
  return imagepts

def loadrecon(path):
    f = open(path)
    reconstruction = json.load(f)
    reconstruction = reconstruction[0]
    f.close()
    return reconstruction

def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    T =-R @ centroid_A + centroid_B
    I = np.array([0,0,0,1]).reshape(1,4)
    Rt = np.concatenate([R.copy().T,np.dot(-1*R.copy().T,T)],axis=-1)
    M  = np.concatenate([Rt,I],axis=0)
    
    addA = np.array([[1]]*A.shape[1])
    addB = np.array([[1]]*B.shape[1])
    
    Aadd = np.concatenate([A.T,addA],axis=1)
    Badd = np.concatenate([B.T,addB],axis=1)
    BtoA = np.apply_along_axis(applyhomogenuos,1,Badd,M)[:,0:3]
    error = A.T-BtoA
    
    print("Error : ",np.average(np.linalg.norm(error,axis=1)))
    
    return R, T




##Load point cloud and relevant parameters

In [ ]:
def findcorrespondence(pcfirst,pcsecond,R1,T1,R2,T2,K1,D1,K2,D2):
    first2d,_ = cv2.projectPoints(pcfirst,R1,T1,K1,D1)
    second2d,_ = cv2.projectPoints(pcsecond,R2,T2,K2,D2)
    first2d = first2d.squeeze()
    second2d = second2d.squeeze()

    # filter xy and with 0 - w and 0 -h range

    filterfirst  = (first2d[:,0]>0) & (first2d[:,0]<1920) & (first2d[:,1]>0) & (first2d[:,1]<1080)
    filtersecond = (second2d[:,0]>0) & (second2d[:,0]<1920) & (second2d[:,1]>0) & (second2d[:,1]<1080)

    print("filterfirst",filterfirst.shape)
    print("filtersecond",filtersecond.shape)

    #apply the created filter

    first2d = first2d[filterfirst]
    second2d = second2d[filtersecond]

    # filter corresponding pc locations to acheive consistence
    print("PC First : ",pcfirst.shape)
    pcfirstfiltered = pcfirst[filterfirst]
    pcsecondfiltered = pcsecond[filtersecond]

    print("after filtering first", first2d.shape)
    print("pc after filtering second", pcfirstfiltered.shape)

    print("after filtering second", second2d.shape)
    print("pc after filtering first", pcsecondfiltered.shape)


    # find nearest corresponding point by reconstruction

    tree = KDTree(second2d)
    dist,ind = tree.query(first2d,k=1)

    # filter pc and get points that are the same in both images

    indexfirst,_ = np.indices((first2d.shape[0],1))

    indexfirst = indexfirst[dist < 1]
    indexsecond = ind[ dist < 1]

    print("Indexes of the first matched points : ",indexsecond[0],indexfirst[0])

    pcfirstfiltered = pcfirstfiltered[indexfirst]

    pcfirstfiltered = pcfirstfiltered.squeeze()

    pcsecondfiltered = pcsecondfiltered[indexsecond]
    
    print("Z stats for fisrt point cloud \n Average : ",np.average(pcfirstfiltered[:,2]))
    print("STD : ",np.std(pcfirstfiltered[:,2]))
    print("Min : ",np.max(pcfirstfiltered[:,2]))
    print("Max : ",np.min(pcfirstfiltered[:,2]))

    print("Z stats for second point cloud \n Average",np.average(pcsecondfiltered[:,2]))
    print("STD : ",np.std(pcsecondfiltered[:,2]))
    print("Min : ",np.max(pcsecondfiltered[:,2]))
    print("Max : ",np.min(pcsecondfiltered[:,2]))

    print(pcfirstfiltered.shape,pcsecondfiltered.shape)
    
    return pcfirstfiltered,pcsecondfiltered



In [ ]:
def findsameimg(sparsefirst,sparsesecond,densefirst,densesecond,image_name):

  K1,D1,w1,h1 = getKD(sparsefirst)
  K2,D2,w2,h2 = getKD(sparsesecond)

  R1,T1 = getRTfromjson(sparsefirst,image_name)
  R2,T2 = getRTfromjson(sparsesecond,image_name)

  print("R1: ",R1) 
  print("R2: ",R2)

  print("T1: ",T1) 
  print("T2: ",T2)

  framefirst,framesecond = findcorrespondence(densefirst,densesecond,R1,T1,R2,T2,K1,D1,K2,D2)

  return framefirst,framesecond



In [ ]:
firstreconpath = "/content/drive/MyDrive/3D Reconstruction/FirstSet/reconstruction.json"
secondreconpath = "/content/drive/MyDrive/3D Reconstruction/SecondSet/reconstruction.json"
firstplypath = '/content/drive/MyDrive/3D Reconstruction/FirstSet/undistorted/depthmaps/merged.ply'
secondplypath = '/content/drive/MyDrive/3D Reconstruction/SecondSet/undistorted/depthmaps/merged.ply'


sparsefirst = loadrecon(firstreconpath)
sparsesecond = loadrecon(secondreconpath)


densefirst,colorfirst = read_ply_xyzrgb(firstplypath)
densesecond,colorsecond = read_ply_xyzrgb(secondplypath)


print("Z stats for fisrt point cloud \n Average : ",np.average(densefirst[:,2]))
print("STD : ",np.std(densefirst[:,2]))
print("Min : ",np.max(densefirst[:,2]))
print("Max : ",np.min(densefirst[:,2]))

print("Z stats for second point cloud \n Average",np.average(densesecond[:,2]))
print("STD : ",np.std(densesecond[:,2]))
print("Min : ",np.max(densesecond[:,2]))
print("Max : ",np.min(densesecond[:,2]))

Z stats for fisrt point cloud 
 Average :  -1.2343696
STD :  2.3083184
Min :  4.784
Max :  -5.7022
Z stats for second point cloud 
 Average 0.6471741
STD :  2.6576564
Min :  7.5664
Max :  -4.4268


Filter the point cloud based on various conditions

Find Correspondence in Multiple images and merge their ordered point cloud

In [ ]:
frame234first ,frame234second =  findsameimg(sparsefirst,sparsesecond,densefirst,densesecond,"Frame234.png")
frame240first ,frame240second = findsameimg(sparsefirst,sparsesecond,densefirst,densesecond,"Frame240.png")
frame246first ,frame246second = findsameimg(sparsefirst,sparsesecond,densefirst,densesecond,"Frame246.png")

pcfirstfiltered = np.concatenate([frame240first,frame246first],axis=0)
pcsecondfiltered = np.concatenate([frame240second,frame246second],axis=0)

print(pcfirstfiltered.shape,pcsecondfiltered.shape)
    

R1:  [[ 0.99864213 -0.05121597  0.00953047]
 [ 0.00185443 -0.14787882 -0.98900375]
 [ 0.05206214  0.98767848 -0.14758305]]
R2:  [[ 0.99968288 -0.02503641 -0.00270547]
 [-0.00739868 -0.18931885 -0.98188779]
 [ 0.02407075  0.98159643 -0.18944405]]
T1:  [[-0.58877738]
 [ 1.94109855]
 [ 9.21078542]]
T2:  [[-0.60214217]
 [-3.66170989]
 [15.29444842]]
filterfirst (445363,)
filtersecond (537926,)
PC First :  (445363, 3)
after filtering first (188458, 2)
pc after filtering second (188458, 3)
after filtering second (383468, 2)
pc after filtering first (383468, 3)
Indexes of the first matched points :  149898 [26]
Z stats for fisrt point cloud 
 Average :  1.6494148
STD :  0.793207
Min :  4.784
Max :  0.2394
Z stats for second point cloud 
 Average -2.940844
STD :  0.8014669
Min :  0.2557
Max :  -4.4268
(77724, 3) (77724, 3)
R1:  [[ 0.99817933 -0.05892719  0.01286897]
 [ 0.00338861 -0.15823464 -0.98739572]
 [ 0.06022077  0.98564161 -0.15774686]]
R2:  [[ 0.999473   -0.0324437   0.00106633]
 [-0.0

Find scaling factor between pointclouds

In [ ]:
#Take thousand sets of corresponding random lines on both point cloud and estimate scaling factor using euclidean distance
import random

randomlines = [random.sample(range(0,max(pcfirstfiltered.shape)),2) for i in range(0,1000)]
randomlines = np.array(randomlines)

startpointsfirst = pcfirstfiltered[randomlines[:,0]]
endpointsfirst = pcfirstfiltered[randomlines[:,1]]

diff1st = startpointsfirst - endpointsfirst
zvalfirst = pcfirstfiltered[:,2]
lengthfirst  = np.linalg.norm(zvalfirst,axis=0) 

startpointssecond = pcsecondfiltered[randomlines[:,0]]
endpointssecond = pcsecondfiltered[randomlines[:,1]]

diff2nd = startpointssecond - endpointssecond
zvalsecond = pcsecondfiltered[:,2]
lengthsecond  =  np.linalg.norm(zvalsecond,axis=0) 
print(lengthsecond.shape)
print(lengthfirst.shape)
scalefactors  =  np.divide(lengthsecond,lengthfirst) 
avgscalefactor = np.average(scalefactors)


print("ScaleFactors : ",scalefactors)
print(startpointsfirst[0],endpointsfirst[0],startpointssecond[0],endpointssecond[0])
print(randomlines.shape)

()
()
ScaleFactors :  1.6861477
[2.1199 4.5999 1.6718] [-0.8412  2.7582  0.9103] [ 1.8895 -1.9633 -2.9586] [-1.1185 -3.9315 -3.6246]
(1000, 2)


Transform the loaded pc

In [ ]:
!pip install open3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 26 kB/s 
     |████████████████████████████████| 3.1 MB 39.2 MB/s 
     |████████████████████████████████| 1.1 MB 40.7 MB/s 
     |████████████████████████████████| 8.8 MB 35.3 MB/s 
     |████████████████████████████████| 423 kB 40.7 MB/s 
     |████████████████████████████████| 344 kB 30.6 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 9.7 MB 36.9 MB/s 
     |████████████████████████████████| 1.6 MB 28.8 MB/s 
     |████████████████████████████████| 1.2 MB 24.0 MB/s 
     |████████████████████████████████| 59 kB 6.0 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 63.9 MB/s 
     |████████████████████████████████| 80 kB 6.8 MB/s 
     |████████████████████████████████| 133 kB 43.7 MB/s 
     |████████████████████████████████|

In [ ]:
import open3d as o3d
R,T = rigid_transform_3D(pcfirstfiltered.T,pcsecondfiltered.T)

# pcfirstscaled = np.multiply(densefirst,avgscalefactor)

I = np.array([0,0,0,1]).reshape(1,4)

Rt = np.concatenate([R,T],axis=-1)
M  = np.concatenate([Rt,I],axis=0)
source = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(densefirst)
target = o3d.geometry.PointCloud()
target.points = o3d.utility.Vector3dVector(densesecond)
threshold = 0.05
print("Rt : ",Rt,"\nShape : ",Rt.shape)
print("M : ",M,"\nShape : ",M.shape)
reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, M,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
M = reg_p2p.transformation
pcfirstfiltered4D =to4D(pcfirstfiltered)

firsttosecondcalcerror = np.apply_along_axis(applyhomogenuos,1,pcfirstfiltered4D,M)[:,0:3]
error = pcsecondfiltered - firsttosecondcalcerror 

print("Error After ICP : ",np.average(np.linalg.norm(error,axis=1)))

pcfirst4D = to4D(densefirst)

firsttosecond = np.apply_along_axis(applyhomogenuos,1,pcfirst4D,M)[:,0:3]

allpts = np.concatenate([densesecond,firsttosecond],axis=0).astype(np.float32)
allptscolor = np.concatenate([colorsecond,colorfirst],axis=0).astype(np.uint8)

print("All pts :",allpts.shape,allpts)
print("All pts Color:",allptscolor.shape)
    
write_pointcloud(allpts,allptscolor,"FirsttoSecond.ply")
   


Error :  0.10656259673158962
Rt :  [[ 0.99965984 -0.02312177  0.01207138 -0.20036086]
 [ 0.02258628  0.9988302   0.04275649 -6.660301  ]
 [-0.01304587 -0.04246929  0.9990126  -4.408397  ]] 
Shape :  (3, 4)
M :  [[ 0.99965984 -0.02312177  0.01207138 -0.20036086]
 [ 0.02258628  0.9988302   0.04275649 -6.66030121]
 [-0.01304587 -0.04246929  0.99901259 -4.4083972 ]
 [ 0.          0.          0.          1.        ]] 
Shape :  (4, 4)
(142479, 3)
Error After ICP :  0.10659849037853376
(445363, 3)
All pts : (983289, 3) [[ -7.5131      7.3325      3.7658   ]
 [ -7.4906      7.3542      3.7682   ]
 [ -7.1353      7.4921      3.7856   ]
 ...
 [  5.207215  -14.297223   -9.802652 ]
 [  5.332692  -14.323736   -9.799374 ]
 [  5.4124575 -14.326051   -9.800395 ]]
All pts Color: (983289, 3)
